<a href="https://colab.research.google.com/github/mhered/robotica_patito_pycon2025/blob/main/notebooks/training_act3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Edited by MH. This Notebook used to add episodes after training the base model.

# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


# ⚠️ Before running any cells, select A100 GPU runtime type (requires making a purchase, the cheapest option of 100 units pay as you go  for ~10€ will be plenty. 1-2hrs of training expected).

## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [ ]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

## More setup
This cell installs `tree` and the git credential helper

In [ ]:
!apt-get -qq update && apt-get -qq install -y tree
!git config --global credential.helper store

## Login onto Weights & Biases and Huggingface Hub
The first cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging. Go to https://wandb.ai/authorize to obtain the API key (you need to sign up first if you haven't already) and paste it on the first cell when requested.

The second cell logs you into the Huggingface Hub to upload checkpoints.
Go to https://huggingface.co/settings/tokens to create a token (you need to sign up first) then create a token. I called it `collab-pycon`. Paste it in the second cell below. Note: for successive runs you need to click on invalidate & refresh the token and paste it again.

Say Yes to adding the token as git credential.

In [ ]:
!wandb login

In [ ]:
!huggingface-cli login

## Training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=USER/REPO_NAME`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `mhered/recording_test`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

Note: If runtime disconnects before a checkpoint is reached and uploaded you cannot resume, lose progress is lost. By default the first checkpoint is fater 20k steps!!
Note: three versions of the train command are provided:
* one to launch training the first time.
* one to resume training when there is still a local `training_state`
* one to warm start from the weights in the last checkpoint saved in the Hub

In [ ]:
# use this to warm start from saved Hub weights after adding more episodes. Here # steps is the number of steps to do
!cd lerobot && lerobot-train \
--dataset.repo_id=mhered/recording-test \
--policy.type=act \
--output_dir=outputs/train/act_so100_pato_test \
--job_name=act_so100_pato_test2 \
--policy.device=cuda \
--wandb.enable=true \
--policy.repo_id=mhered/my_act2 \
--policy.pretrained_path=mhered/my_act \
--steps=20000 \
--save_checkpoint=true \
--save_freq=2000 \
--eval_freq=2000 \
--policy.use_amp=true \
--num_workers=2 \
--policy.push_to_hub=false


## Upload checkpoint to Hugging Face Hub
Now after training is done check the checkpoints created and upload the last checkpoint to the Hugging Face hub at https://huggingface.co/mhered/my_act2/tree/main

In [ ]:
!tree /content/lerobot/outputs/train/act_so100_pato_test/checkpoints

In [ ]:
!huggingface-cli upload mhered/my_act2 \
/content/lerobot/outputs/train/act_so100_pato_test/checkpoints/last/pretrained_model \
--repo-type model --commit-message "Upload last checkpoint"

## Disconnect to avoid consuming compute

In [ ]:
from google.colab import runtime
runtime.unassign()